In [ ]:
# 1 time

# for langchain classes
%pip install --upgrade --quiet  langchain faiss-cpu tiktoken

# to use models from huggingface
%pip install --upgrade --quiet  transformers --quiet

# to read pdf
%pip install pypdf
#%pip install pymupdf
# required to tranform sentences for RAG
%pip install --upgrade --quiet  langchain langchain_community sentence_transformers datasets

!pip install Ollama -qU
%pip install -qU langchain-ollama

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.




[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from operator import itemgetter

from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
#from langchain_core.prompts import ChatPromptTemplate
from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain.chains import RetrievalQA


from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
import ollama
llama3=ollama.pull('llama3')
ollama.pull('nomic-embed-text')

In [ ]:
#https://python.langchain.com/docs/integrations/text_embedding/ollama/
from langchain_ollama import OllamaEmbeddings

# embeddings = OllamaEmbeddings(
#     model="llama3",
# ) # takes million years to when creating embeddings using this
# mxbai-embed-large
# nomic-embed-text
embeddings = OllamaEmbeddings(
    model="nomic-embed-text",
)


In [ ]:
import os
pdf_files_to_process = []
for root, dirs, files in os.walk("../GenAI/data/"):
    pdf_files_to_process.extend([os.path.join(root, file) for file in files if file.lower().endswith(".pdf")])

pdf_files_to_process

In [8]:
#pdf_files_to_process

#remove unwanted files
pdf_files_to_process.pop(-1)

'../GenAI/data/Twelfthphysics1\\nr\\leph1an.pdf'

In [9]:
batch_docs = []

for files in pdf_files_to_process:
  pdfLoader = PyPDFLoader(files)
  documents = pdfLoader.load()
  batch_docs.extend(documents)

In [10]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=100)
docs = text_splitter.split_documents(batch_docs)

In [ ]:
from langchain.indexes import VectorstoreIndexCreator

# FAISS.from_texts # for text data
vectorstore = FAISS.from_documents(
    docs, embedding=embeddings
)

#save vector for later use
vectorstore.save_local("../GenAI/data/vectorstores/faiss/twelfthphysics1")

In [ ]:
# load vector from disk

# loading faiss index from local storage
vectorstore = FAISS.load_local("../GenAI/data/vectorstores/faiss/twelfthphysics1", embeddings,allow_dangerous_deserialization=True)

In [15]:
retriever = vectorstore.as_retriever()

In [17]:
retrieved_documents = retriever.invoke("What is DISPLACEMENT CURRENT?")

In [19]:
# show the retrieved document's content
print(retrieved_documents[0].page_content)

point P outside the plates [Fig. 8.1(a)] is the same as at a point M just
inside, as it should be. The current carried by conductors due to flow of
charges is called conduction current . The current, given by Eq. (8.4), is a
new term, and is due to changing electric field (or electric displacement,
an old term still used sometimes). It is, therefore, called displacement
current  or Maxwell’s displacement current. Figure 8.2 shows the electric


In [ ]:
# https://python.langchain.com/v0.1/docs/integrations/llms/ollama/
from langchain_community.llms import Ollama
llma = Ollama(model="llama3.1")
qa = RetrievalQA.from_chain_type(
    llm=llma, chain_type="stuff", retriever=retriever, return_source_documents=False)

In [ ]:
query = "What is DISPLACEMENT CURRENT? give 10 points from the textbook, do not generate on your own"
answer = qa({"query": query})

In [ ]:
# llama3
print("Answer:",answer.get('result'))

Answer: According to the given context, DISPLACEMENT CURRENT is "a new term" due to changing electric field (or electric displacement), also known as Maxwell's displacement current. It is a type of current that has the same physical effects as conduction current and can be present in different regions of space, including cases where there is no conduction current nearby.


In [33]:
# llama3.1
print("Answer:",answer.get('result'))

Answer: A conduction current carried by conductors due to changing electric field. It is also known as Maxwell's displacement current.


In [34]:
answer

{'query': 'What is DISPLACEMENT CURRENT?',
 'result': "A conduction current carried by conductors due to changing electric field. It is also known as Maxwell's displacement current."}

In [36]:
# llama3.1
print("Answer:",answer.get('result'))

Answer: Based on the provided context, here are 10 points about Displacement Current:

1. The current carried by conductors due to flow of charges is called conduction current.
2. Displacement current is a new term that was introduced by Maxwell.
3. It is caused by changing electric field (or electric displacement).
4. Therefore, it is also known as Maxwell’s displacement current.
5. Displacement current has the same physical effects as the conduction current.
6. In some cases, the displacement current may be zero since the electric field E does not change with time.
7. Both conduction and displacement currents may be present in different regions of space.
8. The prediction of such a displacement current can be verified experimentally.
9. A magnetic field between the plates of a capacitor can be measured and is seen to be the same as that just outside the plates.
10. The laws of electricity and magnetism are consistent with the presence of displacement current, which has far-reaching c